# Account - AGE Checkout PROD

## Database Summary

In [1]:
Import-Module AWS.Tools.Common, AWS.Tools.DynamoDBv2, AWS.Tools.RDS
Import-Module '/Users/conrad.gauntlett/WorkArea/Repos/DBA_MISC/PowerShell/Modules/DevOpsToolkit'

$Account            = 'DevOps-AGE-Checkout-PROD'
$RDSSummaryColumns  = 'AvailabilityZone', 'DBSnapshotIdentifier', 'SnapshotCreateTime', 'SnapshotType', 'Status'
$Profiles           = Get-AWSCredentials -ListProfileDetail | Where-Object { $_.ProfileName -match $Account }
$Regions            = "eu-central-1" , "eu-west-2" , "eu-west-1", "us-east-1"
$Summary            = Get-DoAWSDBInformation $Profiles $Regions
$Summary | Format-Table -AutoSize


Account           AccountID    Region    RDSInstances DynamoDBTables
-------           ---------    ------    ------------ --------------
AGE-Checkout-PROD 004796833142 eu-west-2            0              3



### DynamoDB Tables

#### List Tables

In [13]:
$ddbTables = Get-DoDDBTableInformation $Profiles $Regions 
$ddbTables | Format-Table -AutoSize


Region    TableName                   TableBackup CreationDateTime     ItemCount TableSizeMB
------    ---------                   ----------- ----------------     --------- -----------
eu-west-2 checkout-session                  False 7/15/2022 9:06:38 AM      1833 10 MB
eu-west-2 restricted-party-properties       False 3/7/2023 12:00:20 PM     21648 0 MB
eu-west-2 terraform-locks                   False 2/28/2022 4:18:33 PM        34 0 MB



#### List Backups

In [3]:
$BackupsList    = Get-DDBBackupList -ProfileName $Profiles.ProfileName -Region eu-west-2
$Columns        = 'TableName', 'BackupName', 'BackupCreationDateTime',    'BackupType' #  'BackupStatus','BackupSizeBytes',
$BackupsList | Sort-Object BackupCreationDateTime -Descending | Select-Object -Property $Columns | Format-Table -AutoSize

#### Get the latest backup for a table

In [4]:
$Table = 'terraform-locks'
if($BackupsList){

    $Backup = $BackupsList | Where-Object { $_.TableName -eq $Table } | Sort-Object -Property BackupCreationDateTime -Descending | Select-Object -First 1
    $Bk = Get-DDBBackup -BackupArn $Backup.BackupArn -ProfileName $Profiles.ProfileName
    $Bk.BackupDetails

}

#### Create a new backup for the table

In [6]:
$Table = 'terraform-locks'
$BackupNew = New-DDBBackup -TableName $Table -ProfileName $Profiles.ProfileName -BackupName "$Table-$(get-date -format "yyyy-MM-dd-HH-mm-ss")" -Region eu-west-2
$BackupNew


BackupArn              : arn:aws:dynamodb:eu-west-2:004796833142:table/terraform-locks/backup/01721
                         119182811-cb5ef1e8
BackupCreationDateTime : 7/16/2024 8:39:42 AM
BackupExpiryDateTime   : 1/1/0001 12:00:00 AM
BackupName             : terraform-locks-2024-07-16-09-39-42
BackupSizeBytes        : 4524
BackupStatus           : CREATING
BackupType             : USER



Error: Command failed: SubmitCode: $Table = 'terraform-locks' ...

#### Check new backup details

In [7]:
$Bk = Get-DDBBackup -BackupArn $BackupNew.BackupArn -ProfileName $Profiles.ProfileName -Region eu-west-2
$Bk.BackupDetails


BackupArn              : arn:aws:dynamodb:eu-west-2:004796833142:table/terraform-locks/backup/01721
                         119182811-cb5ef1e8
BackupCreationDateTime : 7/16/2024 8:39:42 AM
BackupExpiryDateTime   : 1/1/0001 12:00:00 AM
BackupName             : terraform-locks-2024-07-16-09-39-42
BackupSizeBytes        : 4524
BackupStatus           : AVAILABLE
BackupType             : USER



#### List all backups

In [8]:
$BackupsList    = Get-DDBBackupList -ProfileName $Profiles.ProfileName -Region eu-west-2
$Columns        = 'TableName', 'BackupName', 'BackupCreationDateTime',    'BackupType' #  'BackupStatus','BackupSizeBytes',
$BackupsList | Sort-Object BackupCreationDateTime -Descending | Select-Object -Property $Columns | Format-Table -AutoSize


TableName       BackupName                          BackupCreationDateTime BackupType
---------       ----------                          ---------------------- ----------
terraform-locks terraform-locks-2024-07-16-09-39-42 7/16/2024 8:39:42 AM   USER



#### Get the oldest backup

In [10]:
$Table = 'terraform-locks'
$BackupOLDEST = $BackupsList | Where-Object { $_.TableName -eq $Table } | Sort-Object -Property BackupCreationDateTime  | Select-Object -First 1
$Bk = Get-DDBBackup -BackupArn $BackupOLDEST.BackupArn -ProfileName $Profiles.ProfileName
$Bk.BackupDetails


BackupArn              : arn:aws:dynamodb:eu-west-2:004796833142:table/terraform-locks/backup/01721
                         119182811-cb5ef1e8
BackupCreationDateTime : 7/16/2024 8:39:42 AM
BackupExpiryDateTime   : 1/1/0001 12:00:00 AM
BackupName             : terraform-locks-2024-07-16-09-39-42
BackupSizeBytes        : 4524
BackupStatus           : AVAILABLE
BackupType             : USER



#### Delete the oldest database

In [11]:
$BackupDelete = Remove-DDBBackup -BackupArn $BackupOLDEST.BackupArn -ProfileName $Profiles.ProfileName -Region eu-west-2 -Confirm:$false
$BackupDelete.BackupDetails


BackupArn              : arn:aws:dynamodb:eu-west-2:004796833142:table/terraform-locks/backup/01721
                         119182811-cb5ef1e8
BackupCreationDateTime : 7/16/2024 8:39:42 AM
BackupExpiryDateTime   : 1/1/0001 12:00:00 AM
BackupName             : terraform-locks-2024-07-16-09-39-42
BackupSizeBytes        : 4524
BackupStatus           : DELETED
BackupType             : USER



#### List all backups

In [12]:
$BackupsList    = Get-DDBBackupList -ProfileName $Profiles.ProfileName -Region eu-west-2
$Columns        = 'TableName', 'BackupName', 'BackupCreationDateTime',    'BackupType' #  'BackupStatus','BackupSizeBytes',
$BackupsList | Sort-Object BackupCreationDateTime -Descending | Select-Object -Property $Columns | Format-Table -AutoSize